# Lowest Income PTC Recipients Analysis
## Impact of IRA PTC Expansion Expiration on Low-Income Households

In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the reform (IRA expiration)
reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")

reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

In [3]:
# Extract key variables for 2026
year = 2026
state = baseline.calculate("state_code", map_to="household", period=year)
num_dependents = baseline.calculate("tax_unit_dependents", map_to="household", period=year)
married = baseline.calculate("is_married", map_to="household", period=year)
employment_income = baseline.calculate("employment_income", map_to="household", period=year)
self_employment_income = baseline.calculate("self_employment_income", map_to="household", period=year)
aca_baseline = baseline.calculate("aca_ptc", map_to="household", period=year)
household_id = baseline.calculate("household_id", map_to="household", period=year)
aca_reform = reformed.calculate("aca_ptc", map_to="household", period=year)

# Create DataFrame
df_outputs = pd.DataFrame({
    "household_id": household_id,
    "State": state,
    "Married": married,
    "Num_Dependents": num_dependents,
    "Employment_Income": employment_income,
    "Self_Employment_Income": self_employment_income,
    "aca_baseline": aca_baseline,
    "aca_reform": aca_reform,
    "weight": aca_baseline.weights
})

df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

## Lowest Income Households with Significant PTC Changes

In [4]:
# Focus on households with reasonable weights for representativeness
MIN_WT = 5_000  # Lower threshold to capture more low-income households

df_big = df_outputs[df_outputs["weight"] >= MIN_WT].copy()

# Calculate weighted changes
df_big["wt_change"] = df_big["net_change"] * df_big["weight"]

# Find lowest income households with PTC gains
N = 20  # Show more examples

cols = ["household_id", "State", "Employment_Income", "Self_Employment_Income", 
        "Married", "Num_Dependents", "weight", "aca_baseline", "aca_reform", "net_change"]

# Filter for households with significant PTC changes
df_with_changes = df_big[df_big["net_change"].abs() > 100]

# Get lowest income households that gain PTC
lowest_income_gainers = df_with_changes[df_with_changes["net_change"] > 0].nsmallest(N, "Employment_Income")[cols]

print("="*80)
print("LOWEST INCOME HOUSEHOLDS WITH PTC GAINS (>$100)")
print("="*80)
display(lowest_income_gainers)

print(f"\nIncome statistics for ALL households with PTC gains >$100:")
gainers = df_with_changes[df_with_changes['net_change'] > 0]
print(f"  Count: {len(gainers)} households")
print(f"  Weighted count: {gainers['weight'].sum():,.0f}")
print(f"  Mean employment income: ${gainers['Employment_Income'].mean():,.0f}")
print(f"  Median employment income: ${gainers['Employment_Income'].median():,.0f}")
print(f"  Mean PTC gain: ${gainers['net_change'].mean():,.2f}")
print(f"  Median PTC gain: ${gainers['net_change'].median():,.2f}")

LOWEST INCOME HOUSEHOLDS WITH PTC GAINS (>$100)


,household_id,State,Employment_Income,Self_Employment_Income,Married,Num_Dependents,weight,aca_baseline,aca_reform,net_change
126,1407,NH,0.0,0.000000,0.0,0.0,7159.027832,6721.821289,7607.067871,885.246582
622,6071,CT,0.0,108265.992188,1.0,0.0,5767.458496,0.000000,9069.158203,9069.158203
910,7644,NY,0.0,0.000000,0.0,0.0,7166.459961,0.000000,521.554199,521.554199
1464,10754,NJ,0.0,0.000000,0.0,0.0,8598.881836,12994.137695,13941.881836,947.744141
1620,11567,NJ,0.0,0.000000,1.0,0.0,22300.699219,4964.321777,7177.249023,2212.927246
1703,12073,NJ,0.0,70372.895508,0.0,1.0,17776.513672,1275.654297,2772.335449,1496.681152
1794,12600,PA,0.0,0.000000,1.0,0.0,10381.871094,15751.487305,17833.683594,2082.196289
1807,12678,PA,0.0,-10825.516602,1.0,0.0,9439.008789,20229.839844,22412.552734,2182.712891
2147,14419,OH,0.0,0.000000,1.0,0.0,8982.781250,17175.761719,18847.708984,1671.947266
2359,15396,OH,0.0,31397.138672,1.0,0.0,18590.898438,19615.958984,21573.222656,1957.263672



Income statistics for ALL households with PTC gains >$100:
  Count: 297 households
  Weighted count: 10,759,920
  Mean employment income: $67,609
  Median employment income: $49,360
  Mean PTC gain: $2,124.26
  Median PTC gain: $1,532.79


## Analysis by Income Brackets

In [5]:
# Define income brackets
income_brackets = [
    (0, 25000, "$0-25K"),
    (25000, 50000, "$25K-50K"),
    (50000, 75000, "$50K-75K"),
    (75000, 100000, "$75K-100K"),
    (100000, 150000, "$100K-150K"),
    (150000, 200000, "$150K-200K"),
    (200000, float('inf'), "$200K+")
]

print("="*80)
print("PTC IMPACT BY INCOME BRACKET")
print("="*80)

for low, high, label in income_brackets:
    bracket_df = df_outputs[(df_outputs['Employment_Income'] >= low) & 
                            (df_outputs['Employment_Income'] < high)]
    
    if len(bracket_df) > 0:
        # Calculate weighted averages
        weights = bracket_df['weight']
        weighted_baseline = (bracket_df['aca_baseline'] * weights).sum() / weights.sum()
        weighted_reform = (bracket_df['aca_reform'] * weights).sum() / weights.sum()
        weighted_change = (bracket_df['net_change'] * weights).sum() / weights.sum()
        
        # Count households gaining PTC
        gaining_ptc = bracket_df[(bracket_df['aca_baseline'] == 0) & (bracket_df['aca_reform'] > 0)]
        
        print(f"\n{label}:")
        print(f"  Households: {len(bracket_df):,}")
        print(f"  Weighted count: {weights.sum():,.0f}")
        print(f"  Avg baseline PTC: ${weighted_baseline:,.2f}")
        print(f"  Avg reform PTC: ${weighted_reform:,.2f}")
        print(f"  Avg change: ${weighted_change:,.2f}")
        print(f"  Households gaining PTC: {len(gaining_ptc)} ({gaining_ptc['weight'].sum():,.0f} weighted)")

PTC IMPACT BY INCOME BRACKET

$0-25K:
  Households: 6,729
  Weighted count: 59,966,238
  Avg baseline PTC: $374.60
  Avg reform PTC: $518.86
  Avg change: $144.27
  Households gaining PTC: 71 (1,113,833 weighted)

$25K-50K:
  Households: 2,908
  Weighted count: 19,040,432
  Avg baseline PTC: $779.11
  Avg reform PTC: $986.98
  Avg change: $207.87
  Households gaining PTC: 35 (155,343 weighted)

$50K-75K:
  Households: 2,610
  Weighted count: 17,163,021
  Avg baseline PTC: $790.77
  Avg reform PTC: $1,117.49
  Avg change: $326.72
  Households gaining PTC: 115 (599,608 weighted)

$75K-100K:
  Households: 2,167
  Weighted count: 11,663,570
  Avg baseline PTC: $657.33
  Avg reform PTC: $914.85
  Avg change: $257.51
  Households gaining PTC: 132 (244,555 weighted)

$100K-150K:
  Households: 2,879
  Weighted count: 14,914,721
  Avg baseline PTC: $411.40
  Avg reform PTC: $745.48
  Avg change: $334.08
  Households gaining PTC: 210 (553,606 weighted)

$150K-200K:
  Households: 1,594
  Weighted

## FPL Analysis for Low-Income Households

In [6]:
# Calculate FPL ratios
# 2026 FPL estimates (rough approximations)
fpl_2026 = {
    1: 15570,   # Single person
    2: 21130,   # Couple  
    3: 26650,   # Family of 3
    4: 32200,   # Family of 4
    5: 37750,   # Family of 5
    6: 43300,   # Family of 6
    7: 48850,   # Family of 7
    8: 54400,   # Family of 8
}

# Calculate household size
df_outputs['household_size'] = df_outputs.apply(
    lambda row: (1 + row['Married'] + row['Num_Dependents']) if not pd.isna(row['Married']) else 1,
    axis=1
)

# Map FPL based on household size
df_outputs['fpl_threshold'] = df_outputs['household_size'].map(lambda x: fpl_2026.get(min(int(x), 8), 54400))
df_outputs['fpl_ratio'] = (df_outputs['Employment_Income'] / df_outputs['fpl_threshold']) * 100

# Analyze by FPL brackets
fpl_brackets = [
    (0, 100, "0-100% FPL"),
    (100, 150, "100-150% FPL"),
    (150, 200, "150-200% FPL"),
    (200, 250, "200-250% FPL"),
    (250, 300, "250-300% FPL"),
    (300, 400, "300-400% FPL"),
    (400, float('inf'), "400%+ FPL")
]

print("="*80)
print("PTC IMPACT BY FPL LEVEL")
print("="*80)

for low, high, label in fpl_brackets:
    bracket_df = df_outputs[(df_outputs['fpl_ratio'] >= low) & 
                            (df_outputs['fpl_ratio'] < high)]
    
    if len(bracket_df) > 0:
        weights = bracket_df['weight']
        weighted_baseline = (bracket_df['aca_baseline'] * weights).sum() / weights.sum()
        weighted_reform = (bracket_df['aca_reform'] * weights).sum() / weights.sum()
        weighted_change = (bracket_df['net_change'] * weights).sum() / weights.sum()
        
        gaining_ptc = bracket_df[(bracket_df['aca_baseline'] == 0) & (bracket_df['aca_reform'] > 0)]
        
        print(f"\n{label}:")
        print(f"  Households: {len(bracket_df):,}")
        print(f"  Weighted count: {weights.sum():,.0f}")
        print(f"  Avg baseline PTC: ${weighted_baseline:,.2f}")
        print(f"  Avg reform PTC: ${weighted_reform:,.2f}")
        print(f"  Avg change: ${weighted_change:,.2f}")
        print(f"  Households gaining new PTC: {len(gaining_ptc)} ({gaining_ptc['weight'].sum():,.0f} weighted)")

PTC IMPACT BY FPL LEVEL

0-100% FPL:
  Households: 6,322
  Weighted count: 56,424,346
  Avg baseline PTC: $383.73
  Avg reform PTC: $533.56
  Avg change: $149.83
  Households gaining new PTC: 69 (1,113,497 weighted)

100-150% FPL:
  Households: 1,083
  Weighted count: 7,849,107
  Avg baseline PTC: $696.40
  Avg reform PTC: $804.16
  Avg change: $107.77
  Households gaining new PTC: 11 (4,212 weighted)

150-200% FPL:
  Households: 1,328
  Weighted count: 6,890,697
  Avg baseline PTC: $689.98
  Avg reform PTC: $878.20
  Avg change: $188.22
  Households gaining new PTC: 15 (13,543 weighted)

200-250% FPL:
  Households: 1,347
  Weighted count: 8,361,869
  Avg baseline PTC: $595.99
  Avg reform PTC: $813.01
  Avg change: $217.02
  Households gaining new PTC: 30 (161,860 weighted)

250-300% FPL:
  Households: 1,257
  Weighted count: 7,051,522
  Avg baseline PTC: $1,039.04
  Avg reform PTC: $1,528.84
  Avg change: $489.80
  Households gaining new PTC: 31 (115,735 weighted)

300-400% FPL:
  Ho

## Characteristics of Very Low Income PTC Recipients

In [7]:
# Focus on very low income households (bottom 20%)
income_20th_percentile = df_outputs['Employment_Income'].quantile(0.2)
very_low_income = df_outputs[df_outputs['Employment_Income'] <= income_20th_percentile]

print("="*80)
print(f"VERY LOW INCOME HOUSEHOLDS (Bottom 20% - Under ${income_20th_percentile:,.0f})")
print("="*80)

# Overall statistics
print(f"\nTotal households: {len(very_low_income):,}")
print(f"Weighted count: {very_low_income['weight'].sum():,.0f}")

# PTC coverage
has_baseline_ptc = very_low_income[very_low_income['aca_baseline'] > 0]
has_reform_ptc = very_low_income[very_low_income['aca_reform'] > 0]
gains_ptc = very_low_income[(very_low_income['aca_baseline'] == 0) & (very_low_income['aca_reform'] > 0)]

print(f"\nPTC Coverage:")
print(f"  With baseline PTC: {len(has_baseline_ptc)} ({has_baseline_ptc['weight'].sum():,.0f} weighted)")
print(f"  With reform PTC: {len(has_reform_ptc)} ({has_reform_ptc['weight'].sum():,.0f} weighted)")
print(f"  Newly gaining PTC: {len(gains_ptc)} ({gains_ptc['weight'].sum():,.0f} weighted)")

# Average benefits
if len(has_baseline_ptc) > 0:
    weighted_baseline = (has_baseline_ptc['aca_baseline'] * has_baseline_ptc['weight']).sum() / has_baseline_ptc['weight'].sum()
    print(f"\nAverage baseline PTC (among recipients): ${weighted_baseline:,.2f}")

if len(has_reform_ptc) > 0:
    weighted_reform = (has_reform_ptc['aca_reform'] * has_reform_ptc['weight']).sum() / has_reform_ptc['weight'].sum()
    print(f"Average reform PTC (among recipients): ${weighted_reform:,.2f}")

if len(gains_ptc) > 0:
    weighted_new_ptc = (gains_ptc['aca_reform'] * gains_ptc['weight']).sum() / gains_ptc['weight'].sum()
    print(f"Average PTC for new recipients: ${weighted_new_ptc:,.2f}")

# Family composition
print(f"\nFamily Composition:")
print(f"  Married households: {very_low_income['Married'].sum()} ({(very_low_income['Married'].mean()*100):.1f}%)")
print(f"  Average dependents: {very_low_income['Num_Dependents'].mean():.2f}")

# State distribution of very low income PTC gainers
if len(gains_ptc) > 0:
    print(f"\nTop 5 states for new low-income PTC recipients:")
    state_counts = gains_ptc.groupby('State')['weight'].sum().sort_values(ascending=False).head(5)
    for state, count in state_counts.items():
        print(f"  {state}: {count:,.0f} weighted households")

VERY LOW INCOME HOUSEHOLDS (Bottom 20% - Under $0)

Total households: 4,501
Weighted count: 40,040,300

PTC Coverage:
  With baseline PTC: 234 (1,533,070 weighted)
  With reform PTC: 287 (2,114,831 weighted)
  Newly gaining PTC: 53 (581,761 weighted)

Average baseline PTC (among recipients): $9,765.06
Average reform PTC (among recipients): $8,478.88
Average PTC for new recipients: $1,287.93

Family Composition:
  Married households: 1261.0 (28.0%)
  Average dependents: 0.13

Top 5 states for new low-income PTC recipients:
  TX: 308,151 weighted households
  CA: 136,999 weighted households
  TN: 64,229 weighted households
  NY: 14,393 weighted households
  IA: 7,342 weighted households


## Specific Examples of Low-Income Households

In [8]:
# Show detailed examples of very low income households affected
print("="*80)
print("DETAILED EXAMPLES OF LOW-INCOME HOUSEHOLDS AFFECTED")
print("="*80)

# Households with zero employment income gaining PTC
zero_income_gainers = df_outputs[(df_outputs['Employment_Income'] == 0) & 
                                 (df_outputs['aca_baseline'] == 0) & 
                                 (df_outputs['aca_reform'] > 0)]

if len(zero_income_gainers) > 0:
    print("\nHouseholds with ZERO employment income gaining PTC:")
    examples = zero_income_gainers.head(10)[['household_id', 'State', 'Self_Employment_Income',
                                             'Married', 'Num_Dependents', 'aca_reform', 'weight']]
    display(examples)
    print(f"Total: {len(zero_income_gainers)} households ({zero_income_gainers['weight'].sum():,.0f} weighted)")

# Households under $10K gaining significant PTC
under_10k = df_outputs[(df_outputs['Employment_Income'] < 10000) & 
                       (df_outputs['Employment_Income'] > 0) &
                       (df_outputs['net_change'] > 1000)]

if len(under_10k) > 0:
    print("\nHouseholds with <$10K income gaining >$1000 in PTC:")
    examples = under_10k.nlargest(10, 'net_change')[['household_id', 'State', 'Employment_Income',
                                                      'aca_baseline', 'aca_reform', 'net_change', 'weight']]
    display(examples)
    print(f"Total: {len(under_10k)} households ({under_10k['weight'].sum():,.0f} weighted)")

DETAILED EXAMPLES OF LOW-INCOME HOUSEHOLDS AFFECTED

Households with ZERO employment income gaining PTC:


,household_id,State,Self_Employment_Income,Married,Num_Dependents,aca_reform,weight
622,6071,CT,108265.992188,1.0,0.0,9069.158203,5767.458496
882,7400,NY,0.000000,0.0,0.0,2666.810547,960.255676
903,7564,NY,70372.898438,0.0,0.0,2494.150391,2329.417236
910,7644,NY,0.000000,0.0,0.0,521.554199,7166.459961
940,7786,NY,81199.500000,0.0,0.0,1638.903809,2846.937500
1029,8230,NY,189465.500000,1.0,6.0,4742.173828,936.465210
1346,10005,NY,43306.398438,1.0,0.0,6065.594727,153.783966
2144,14395,OH,108268.164062,1.0,0.0,12775.871094,2511.855713
3739,22352,WI,40058.417969,0.0,0.0,2499.087891,1064.238159
4252,24709,IA,0.000000,1.0,0.0,3870.446289,6387.240723


Total: 53 households (581,761 weighted)

Households with <$10K income gaining >$1000 in PTC:


,household_id,State,Employment_Income,aca_baseline,aca_reform,net_change,weight
5229,33105,DC,548.445068,0.000000,9024.396484,9024.396484,750.579529
16938,131075,GA,4231.810547,0.000000,6614.862793,6614.862793,0.000061
15484,112920,MN,8006.647980,5480.122559,7668.104492,2187.981934,0.000110
18620,149209,TX,8019.194092,0.000000,2102.861328,2102.861328,0.002293
13317,96320,NY,6783.917480,0.000000,2090.177734,2090.177734,0.000791
4310,24983,MO,1099.083862,3736.214111,5794.061523,2057.847412,1068.922119
12759,88965,ME,8169.807861,26543.898438,28568.738281,2024.839844,0.378303
9949,68376,ID,3510.048340,7120.457031,9006.253906,1885.796875,2367.052734
10543,73619,AZ,9872.010742,1281.723145,3166.381348,1884.658203,2231.716309
4021,23542,MN,3290.670166,0.000000,1835.048828,1835.048828,795.783264


Total: 33 households (30,630 weighted)


## Summary Statistics

In [9]:
print("="*80)
print("SUMMARY: IMPACT ON LOW-INCOME HOUSEHOLDS")
print("="*80)

# Define low income as bottom 40%
income_40th_percentile = df_outputs['Employment_Income'].quantile(0.4)
low_income = df_outputs[df_outputs['Employment_Income'] <= income_40th_percentile]

print(f"\nLow-income threshold (40th percentile): ${income_40th_percentile:,.0f}")
print(f"Number of low-income households: {len(low_income):,}")
print(f"Weighted count: {low_income['weight'].sum():,.0f}")

# Calculate weighted averages for low-income households
weights = low_income['weight']
weighted_baseline = (low_income['aca_baseline'] * weights).sum() / weights.sum()
weighted_reform = (low_income['aca_reform'] * weights).sum() / weights.sum()
weighted_change = (low_income['net_change'] * weights).sum() / weights.sum()

print(f"\nAverage PTC amounts for low-income households:")
print(f"  Baseline (with IRA): ${weighted_baseline:,.2f}")
print(f"  Reform (IRA expired): ${weighted_reform:,.2f}")
print(f"  Net change: ${weighted_change:,.2f}")

# Coverage changes
gaining = low_income[(low_income['aca_baseline'] == 0) & (low_income['aca_reform'] > 0)]
losing = low_income[(low_income['aca_baseline'] > 0) & (low_income['aca_reform'] == 0)]
keeping = low_income[(low_income['aca_baseline'] > 0) & (low_income['aca_reform'] > 0)]

print(f"\nCoverage changes among low-income households:")
print(f"  Gaining PTC: {len(gaining)} households ({gaining['weight'].sum():,.0f} weighted)")
print(f"  Losing PTC: {len(losing)} households ({losing['weight'].sum():,.0f} weighted)")
print(f"  Keeping PTC: {len(keeping)} households ({keeping['weight'].sum():,.0f} weighted)")

if len(gaining) > 0:
    avg_gain = (gaining['aca_reform'] * gaining['weight']).sum() / gaining['weight'].sum()
    print(f"\n  Average PTC for those gaining: ${avg_gain:,.2f}")

if len(keeping) > 0:
    avg_increase = (keeping['net_change'] * keeping['weight']).sum() / keeping['weight'].sum()
    print(f"  Average increase for those keeping PTC: ${avg_increase:,.2f}")

# Total impact
total_baseline = (low_income['aca_baseline'] * weights).sum()
total_reform = (low_income['aca_reform'] * weights).sum()
print(f"\nTotal PTC spending on low-income households:")
print(f"  Baseline: ${total_baseline/1e9:.2f} billion")
print(f"  Reform: ${total_reform/1e9:.2f} billion")
print(f"  Change: ${(total_reform - total_baseline)/1e9:.2f} billion")

SUMMARY: IMPACT ON LOW-INCOME HOUSEHOLDS

Low-income threshold (40th percentile): $38,933
Number of low-income households: 8,443
Weighted count: 70,712,901

Average PTC amounts for low-income households:
  Baseline (with IRA): $448.43
  Reform (IRA expired): $600.43
  Net change: $152.00

Coverage changes among low-income households:
  Gaining PTC: 88 households (1,198,778 weighted)
  Losing PTC: 0 households (0 weighted)
  Keeping PTC: 727 households (3,823,616 weighted)

  Average PTC for those gaining: $4,672.47
  Average increase for those keeping PTC: $1,346.19

Total PTC spending on low-income households:
  Baseline: $31.71 billion
  Reform: $42.46 billion
  Change: $10.75 billion


In [10]:
# Find household 7786 and extract all relevant details
household_id_target = 7786

# Get the household data
hh_data = df_outputs[df_outputs['household_id'] == household_id_target]

if len(hh_data) > 0:
    print("="*80)
    print(f"HOUSEHOLD {household_id_target} DETAILS")
    print("="*80)
    
    # Basic info from dataframe
    row = hh_data.iloc[0]
    print("\nBasic Information:")
    print(f"  State: {row['State']}")
    print(f"  Employment Income: ${row['Employment_Income']:,.2f}")
    print(f"  Self-Employment Income: ${row['Self_Employment_Income']:,.2f}")
    print(f"  Married: {bool(row['Married'])}")
    print(f"  Number of Dependents: {int(row['Num_Dependents'])}")
    print(f"  Baseline PTC (2026): ${row['aca_baseline']:,.2f}")
    print(f"  Reform PTC (2026): ${row['aca_reform']:,.2f}")
    print(f"  Weight: {row['weight']:,.2f}")
    
    # Now get more detailed information from the simulation
    print("\n" + "="*80)
    print("EXTRACTING DETAILED HOUSEHOLD INFORMATION FROM SIMULATION")
    print("="*80)
else:
    print(f"Household {household_id_target} not found in outputs")

HOUSEHOLD 7786 DETAILS

Basic Information:
  State: NY
  Employment Income: $0.00
  Self-Employment Income: $81,199.50
  Married: False
  Number of Dependents: 0
  Baseline PTC (2026): $0.00
  Reform PTC (2026): $1,638.90
  Weight: 2,846.94

EXTRACTING DETAILED HOUSEHOLD INFORMATION FROM SIMULATION


In [11]:
# Get detailed information for household 7786 from baseline simulation
year = 2026
hh_id = 7786

# Find the index for this household
hh_mask = baseline.calculate("household_id", map_to="person", period=year) == hh_id

# Get person-level details
ages = baseline.calculate("age", map_to="person", period=year)[hh_mask]
person_ids = baseline.calculate("person_id", map_to="person", period=year)[hh_mask]
employment_inc_person = baseline.calculate("employment_income", map_to="person", period=year)[hh_mask]
self_emp_inc_person = baseline.calculate("self_employment_income", map_to="person", period=year)[hh_mask]

print("Person-level details:")
print("-" * 40)
for i, (pid, age, emp, self_emp) in enumerate(zip(person_ids, ages, employment_inc_person, self_emp_inc_person)):
    print(f"\nPerson {i+1} (ID: {pid}):")
    print(f"  Age: {age}")
    print(f"  Employment income: ${emp:,.2f}")
    print(f"  Self-employment income: ${self_emp:,.2f}")

# Get tax unit information
tu_mask = baseline.calculate("household_id", map_to="tax_unit", period=year) == hh_id
tax_unit_ids = baseline.calculate("tax_unit_id", map_to="tax_unit", period=year)[tu_mask]

print("\n" + "="*40)
print("Tax Unit Information:")
print("-" * 40)

for tu_id in tax_unit_ids:
    # Get tax unit specific variables
    tu_specific_mask = baseline.calculate("tax_unit_id", map_to="tax_unit", period=year) == tu_id
    
    # Get MAGI and related variables
    magi = baseline.calculate("ma_aca_magi", map_to="tax_unit", period=year)[tu_specific_mask]
    is_ptc_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="tax_unit", period=year)[tu_specific_mask]
    
    print(f"\nTax Unit ID: {tu_id}")
    print(f"  MA ACA MAGI: ${magi[0]:,.2f}")
    print(f"  Is PTC eligible (baseline): {bool(is_ptc_eligible[0])}")

Person-level details:
----------------------------------------

Person 1 (ID: 2411):
  Age: 56.0
  Employment income: $0.00
  Self-employment income: $81,199.50

Tax Unit Information:
----------------------------------------


ValueError: Variable ma_aca_magi does not exist.

In [12]:
# Try with correct variable names
year = 2026
hh_id = 7786

# Get tax unit information
tu_mask = baseline.calculate("household_id", map_to="tax_unit", period=year) == hh_id
tax_unit_ids = baseline.calculate("tax_unit_id", map_to="tax_unit", period=year)[tu_mask]

print("Tax Unit Information:")
print("-" * 40)

for tu_id in tax_unit_ids:
    # Get tax unit specific variables
    tu_specific_mask = baseline.calculate("tax_unit_id", map_to="tax_unit", period=year) == tu_id
    
    # Get MAGI and related variables
    aca_magi = baseline.calculate("aca_magi", map_to="tax_unit", period=year)[tu_specific_mask]
    is_ptc_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="tax_unit", period=year)[tu_specific_mask]
    slcsp = baseline.calculate("slcsp", map_to="tax_unit", period=year)[tu_specific_mask]
    aca_ptc = baseline.calculate("aca_ptc", map_to="tax_unit", period=year)[tu_specific_mask]
    
    print(f"\nTax Unit ID: {tu_id}")
    print(f"  ACA MAGI: ${aca_magi[0]:,.2f}")
    print(f"  Is PTC eligible (baseline): {bool(is_ptc_eligible[0])}")
    print(f"  SLCSP: ${slcsp[0]:,.2f}")
    print(f"  ACA PTC (baseline): ${aca_ptc[0]:,.2f}")
    
    # Get additional details
    aca_magi_fraction = baseline.calculate("aca_magi_fraction", map_to="tax_unit", period=year)[tu_specific_mask]
    phase_out_rate = baseline.calculate("aca_ptc_phase_out_rate", map_to="tax_unit", period=year)[tu_specific_mask]
    
    print(f"  ACA MAGI fraction of FPL: {aca_magi_fraction[0]:.3f}")
    print(f"  PTC phase out rate: {phase_out_rate[0]:.3f}")

# Get household location details
hh_specific_mask = baseline.calculate("household_id", map_to="household", period=year) == hh_id
state_fips = baseline.calculate("state_fips", map_to="household", period=year)[hh_specific_mask]
county = baseline.calculate("county", map_to="household", period=year)[hh_specific_mask]
three_digit_zip = baseline.calculate("three_digit_zip_code", map_to="household", period=year)[hh_specific_mask]
rating_area = baseline.calculate("slcsp_rating_area", map_to="household", period=year)[hh_specific_mask]

print("\n" + "="*40)
print("Location Information:")
print("-" * 40)
print(f"  State FIPS: {state_fips[0]}")
print(f"  County: {county[0]}")
print(f"  Three-digit ZIP: {three_digit_zip[0]}")
print(f"  Rating area: {rating_area[0]}")

Tax Unit Information:
----------------------------------------

Tax Unit ID: 778601


KeyError: 0

In [13]:
# Get complete details for household 7786
year = 2026
hh_id = 7786

# Get household mask
hh_mask = baseline.calculate("household_id", map_to="household", period=year) == hh_id

# Get all household IDs to find the index
all_hh_ids = baseline.calculate("household_id", map_to="household", period=year)
hh_index = np.where(all_hh_ids == hh_id)[0]

if len(hh_index) > 0:
    print("="*80)
    print(f"COMPLETE HOUSEHOLD {hh_id} DETAILS FOR INTEGRATION TEST")
    print("="*80)
    
    # Get person-level information
    person_hh_ids = baseline.calculate("household_id", map_to="person", period=year)
    person_mask = person_hh_ids == hh_id
    
    # Extract person details
    ages = baseline.calculate("age", map_to="person", period=year)[person_mask].values
    emp_income = baseline.calculate("employment_income", map_to="person", period=year)[person_mask].values
    self_emp_income = baseline.calculate("self_employment_income", map_to="person", period=year)[person_mask].values
    is_eshi_eligible = baseline.calculate("is_aca_eshi_eligible", map_to="person", period=year)[person_mask].values
    
    print("\n1. PERSON-LEVEL DETAILS:")
    print("-" * 40)
    for i in range(len(ages)):
        print(f"Person {i+1}:")
        print(f"  age: {int(ages[i])}")
        print(f"  employment_income: {emp_income[i]:,.0f}")
        print(f"  self_employment_income: {self_emp_income[i]:,.0f}")
        print(f"  is_aca_eshi_eligible: {str(is_eshi_eligible[i]).lower()}")
    
    # Get tax unit information
    tu_hh_ids = baseline.calculate("household_id", map_to="tax_unit", period=year)
    tu_mask = tu_hh_ids == hh_id
    
    # Get tax unit variables (use values array to avoid indexing issues)
    aca_magi = baseline.calculate("aca_magi", map_to="tax_unit", period=year)[tu_mask].values
    is_ptc_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="tax_unit", period=year)[tu_mask].values
    slcsp = baseline.calculate("slcsp", map_to="tax_unit", period=year)[tu_mask].values
    aca_ptc_base = baseline.calculate("aca_ptc", map_to="tax_unit", period=year)[tu_mask].values
    magi_fraction = baseline.calculate("aca_magi_fraction", map_to="tax_unit", period=year)[tu_mask].values
    phase_out = baseline.calculate("aca_ptc_phase_out_rate", map_to="tax_unit", period=year)[tu_mask].values
    
    print("\n2. TAX UNIT DETAILS:")
    print("-" * 40)
    if len(aca_magi) > 0:
        print(f"  aca_magi: {aca_magi[0]:,.2f}")
        print(f"  aca_magi_fraction: {magi_fraction[0]:.3f}")
        print(f"  is_aca_ptc_eligible (baseline): {str(is_ptc_eligible[0]).lower()}")
        print(f"  slcsp: {slcsp[0]:,.2f}")
        print(f"  aca_ptc_phase_out_rate (baseline): {phase_out[0]:.3f}")
        print(f"  aca_ptc (baseline 2026): {aca_ptc_base[0]:,.2f}")
    
    # Get household location
    state_fips = baseline.calculate("state_fips", map_to="household", period=year)[hh_mask].values
    county = baseline.calculate("county", map_to="household", period=year)[hh_mask].values  
    three_digit_zip = baseline.calculate("three_digit_zip_code", map_to="household", period=year)[hh_mask].values
    
    print("\n3. HOUSEHOLD LOCATION:")
    print("-" * 40)
    if len(state_fips) > 0:
        print(f"  state_fips: {int(state_fips[0])}")
        print(f"  county: {county[0]}")
        print(f"  three_digit_zip_code: {int(three_digit_zip[0])}")
    
    # Now get reform values
    aca_ptc_reform = reformed.calculate("aca_ptc", map_to="tax_unit", period=year)[tu_mask].values
    is_ptc_eligible_reform = reformed.calculate("is_aca_ptc_eligible", map_to="tax_unit", period=year)[tu_mask].values
    phase_out_reform = reformed.calculate("aca_ptc_phase_out_rate", map_to="tax_unit", period=year)[tu_mask].values
    
    print("\n4. REFORM (IRA EXPIRED) VALUES:")
    print("-" * 40)
    if len(aca_ptc_reform) > 0:
        print(f"  is_aca_ptc_eligible (reform): {str(is_ptc_eligible_reform[0]).lower()}")
        print(f"  aca_ptc_phase_out_rate (reform): {phase_out_reform[0]:.3f}")
        print(f"  aca_ptc (reform 2026): {aca_ptc_reform[0]:,.2f}")
        print(f"  net_change: {aca_ptc_reform[0] - aca_ptc_base[0]:,.2f}")
else:
    print(f"Household {hh_id} not found")

COMPLETE HOUSEHOLD 7786 DETAILS FOR INTEGRATION TEST

1. PERSON-LEVEL DETAILS:
----------------------------------------
Person 1:
  age: 56
  employment_income: 0
  self_employment_income: 81,200
  is_aca_eshi_eligible: false

2. TAX UNIT DETAILS:
----------------------------------------
  aca_magi: 75,462.94
  aca_magi_fraction: 4.820
  is_aca_ptc_eligible (baseline): 0.0
  slcsp: 0.00
  aca_ptc_phase_out_rate (baseline): 0.095
  aca_ptc (baseline 2026): 0.00


AttributeError: 'NoneType' object has no attribute 'state'